# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

## Introduction

<table><tr>
    <td>
        <img src='misc/mono.png' width="600 px">
        <figcaption>Higley et al., Rev. Sci. Instrum.  87, 033110 (2016)</figcaption>
    </td>
    <td>
        <img src='misc/illustration.png' width="920 px">
    </td>
    <td>
        <img src='misc/expected_data.png' width="800 px">
        <figcaption>Higley et al., submitted</figcaption></td>
</tr></table>

## Data Analysis

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from karabo_data import RunDirectory

In [ ]:
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0095"
run = RunDirectory(run_folder)

run.info()

## Sanity check

*check whether the pulse energy in the pipeline data is calibrated*

In [ ]:
xgm_device = 'SCS_BLU_XGM/XGM/DOOCS'
# xgm_device = 'SA3_XTD10_XGM/XGM/DOOCS'

In [ ]:
xgm_run = run.get_dataframe(fields=[(xgm_device, '*value')])
xgm_run.rename(columns=lambda x: x.split('/')[-1], inplace=True)

idx = 0
tid, data = run.train_from_index(idx)
pipeline_data = data[xgm_device + ':output']

pulse_energy_train_resolved = xgm_run['pulseEnergy.photonFlux'].iloc[idx]
pulse_energy_pulse_resolved = pipeline_data['data.intensityTD'][
    0:int(xgm_run['pulseEnergy.nummberOfBrunches'].iloc[idx])].mean()

print("Train resolved pulse energy is: {:.4f} micro J".format(pulse_energy_train_resolved))
print("Pulse resolved pulse energy is: {:.4f} micro J".format(pulse_energy_pulse_resolved))
if abs(pulse_energy_train_resolved - pulse_energy_pulse_resolved) > 0.01*pulse_energy_train_resolved:
    print("Warning: pulse resolved pulse energy is not calibrated!")

*check the number of pulses and train resolved pulse energy throughtout the run*

In [ ]:
# plot train resolved pulse energy and number of pulses in each train

_, ax = plt.subplots(figsize=(12, 6))
ax2 = ax.twinx()

ln1 = ax.plot(xgm_run['pulseEnergy.nummberOfBrunches'], label="number of pulses", c='g')
ln2 = ax2.plot(xgm_run['pulseEnergy.photonFlux'], label=r"pulse energy ($\mu$J)")

lns = ln1 + ln2
lables = [l.get_label() for l in lns]
ax.legend(lns, lables, loc=4)
ax.set_xlabel("Train ID")
ax.set_ylabel("Number of pulses")
ax2.set_ylabel(r"Pulse energy ($\mu$J)")

## Data analysis

In [ ]:
def MCPintegration(mcptrace, bkgstart, bkgend, intstart, intend):
    """ Simple MCP trace peak integration
    """
    bkg = np.median(mcptrace[bkgstart:bkgend])
    return np.trapz(mcptrace[intstart:intend] - bkg)

In [ ]:
def loadrun(runNB, proposalNB, strangeNB, topic='SCS'):
    """ Load a run and extract the data. Output is a rec array. 
    """
    runFolder = '/gpfs/exfel/exp/{}/{:d}/p{:d}/raw/r{:04d}/'.format(topic, strangeNB, proposalNB, runNB)
    run = RunDirectory(runFolder)
    #run.info()
    
    Nevts = len(run.train_ids)
    
    # return type of the structured array
    names_list = ['train_id', 'mono_nrj', 'SCS_XGM', 'MCP1', 'MCP2', 'MCP3', 'MCP4']
    
    rundata = np.empty((Nevts, len(names_list)))
    
    for k,(tid, data) in enumerate(run.trains()):
        if k == 1:
           break

        print(data['SCS_BLU_XGM/XGM/DOOCS:output']['data.intensityTD'])
        
        SCS_XGM = data['SCS_BLU_XGM/XGM/DOOCS:output']['data.intensityTD'][1]
        MCP1 = MCPintegration(data['SCS_UTC1_MCP/ADC/1:channel_9.output']['data.rawData'],
                             bkgstart, bkgend, intstart, intend)
        MCP2 = MCPintegration(data['SCS_UTC1_MCP/ADC/1:channel_8.output']['data.rawData'],
                             bkgstart, bkgend, intstart, intend)
        MCP3 = MCPintegration(data['SCS_UTC1_MCP/ADC/1:channel_7.output']['data.rawData'],
                             bkgstart, bkgend, intstart, intend)
        MCP4 = MCPintegration(data['SCS_UTC1_MCP/ADC/1:channel_6.output']['data.rawData'],
                             bkgstart, bkgend, intstart, intend)
        nrj = nrjs[runNB]
        
        print(nrj, MCP1, SCS_XGM)
        
        rundata[k,:] = [tid, nrj, SCS_XGM, MCP1, MCP2, MCP3, MCP4]
        k += 1
        
    return np.rec.fromarrays(rundata.T, names=names_list)

In [ ]:
bkgstart = 1000
bkgend = 1500

# narrow negative part integration
intstart = 1584
intend = 1594

topic = 'SCS'
strange = 201831
proposal = 900048
run = 95
nrjs = {95:704.5}

data = loadrun(run, proposal, strange, topic)
print(data)